In [3]:
import numpy as np
import pandas as pd

Matplotlib is building the font cache using fc-list. This may take a moment.


In [74]:
ratings = pd.read_csv('./dataset/movie_rating.csv')

In [75]:
ratings.head()

,critic,title,rating
0,Jack Matthews,Lady in the Water,3.0
1,Jack Matthews,Snakes on a Plane,4.0
2,Jack Matthews,You Me and Dupree,3.5
3,Jack Matthews,Superman Returns,5.0
4,Jack Matthews,The Night Listener,3.0


In [76]:
movie_ratings = ratings.set_index(['title', 'critic']).unstack()
movie_ratings.columns = [col[1] for col in movie_ratings.columns]
movie_ratings

,Claudia Puig,Gene Seymour,Jack Matthews,Lisa Rose,Mick LaSalle,Toby
title,,,,,,
Just My Luck,3.0,1.5,NaN,3.0,2.0,NaN
Lady in the Water,NaN,3.0,3.0,2.5,3.0,NaN
Snakes on a Plane,3.5,3.5,4.0,3.5,4.0,4.5
Superman Returns,4.0,5.0,5.0,3.5,3.0,4.0
The Night Listener,4.5,3.0,3.0,3.0,3.0,NaN
You Me and Dupree,2.5,3.5,3.5,2.5,2.0,1.0


In [77]:
# 위 표에서 유저간 similarity를 구해줌
sim_users = movie_ratings.corr()

In [80]:
sim_users

,Claudia Puig,Gene Seymour,Jack Matthews,Lisa Rose,Mick LaSalle,Toby
Claudia Puig,1.000000,0.314970,0.028571,0.566947,0.566947,0.893405
Gene Seymour,0.314970,1.000000,0.963796,0.396059,0.411765,0.381246
Jack Matthews,0.028571,0.963796,1.000000,0.747018,0.211289,0.662849
Lisa Rose,0.566947,0.396059,0.747018,1.000000,0.594089,0.991241
Mick LaSalle,0.566947,0.411765,0.211289,0.594089,1.000000,0.924473
Toby,0.893405,0.381246,0.662849,0.991241,0.924473,1.000000


In [78]:
# Toby와의 유사도
sim_Toby = sim_users.loc[:, 'Toby'].reset_index()
sim_Toby.columns = ['critic', 'similarity']

In [79]:
sim_Toby

,critic,similarity
0,Claudia Puig,0.893405
1,Gene Seymour,0.381246
2,Jack Matthews,0.662849
3,Lisa Rose,0.991241
4,Mick LaSalle,0.924473
5,Toby,1.000000


In [81]:
# Toby의 점수만 보여줌
rating_critic = movie_ratings.loc[:, 'Toby']

# Toby의 평점이 없는 영화들을 뽑아냄
titles_na_critic = rating_critic[rating_critic.isna()].index.tolist()

# Toby가 평점을 주지 않은 영화들의 제목과 평점, 평론자를 뽑아냄
ratings_t = ratings[ratings.title.isin(titles_na_critic)]

# similarity와 위에서 뽑은 표를 inner join
ratings_t = pd.merge(ratings_t, sim_Toby, how='inner', left_on='critic', right_on='critic')

# 각 평가자의 점수와 Toby와의 유사도를 계산해서 sim_rating이라 넣어줌
ratings_t['sim_rating'] = ratings_t.rating * ratings_t.similarity

# 영화 제목별로 그룹을 지은뒤
# sim_rating의 합, 유사도의 합을 구함
result = ratings_t.groupby(by = 'title').agg({'sim_rating' : 'sum', 'similarity' : 'sum'})

# sim_rating의 합을 유사도의 합으로 나누어 mean_rating을 구해줌
result['mean_rating'] = result.sim_rating / result.similarity

# movie_rating에서 Toby의 평점이 없던 부분을 채워줌
movie_ratings.loc[titles_na_critic, 'Toby'] = result.mean_rating

In [83]:
# Toby의 점수가 채워짐
movie_ratings

,Claudia Puig,Gene Seymour,Jack Matthews,Lisa Rose,Mick LaSalle,Toby
title,,,,,,
Just My Luck,3.0,1.5,NaN,3.0,2.0,2.530981
Lady in the Water,NaN,3.0,3.0,2.5,3.0,2.832550
Snakes on a Plane,3.5,3.5,4.0,3.5,4.0,4.500000
Superman Returns,4.0,5.0,5.0,3.5,3.0,4.000000
The Night Listener,4.5,3.0,3.0,3.0,3.0,3.347790
You Me and Dupree,2.5,3.5,3.5,2.5,2.0,1.000000


In [84]:
def generateRecommendations(userID) :
    global movie_ratings, sim_users
    sim_userID = sim_users.loc[:, userID].reset_index()
    sim_userID.columns = ['critic', 'similarity']
    
    rating_critic = movie_ratings.loc[:, userID]
    titles_na_critic = rating_critic[rating_critic.isna()].index.tolist()
    ratings_t = ratings[ratings.title.isin(titles_na_critic)]
    
    ratings_t = pd.merge(ratings_t, sim_Toby, how='inner', left_on='critic', right_on='critic')
    ratings_t['sim_rating'] = ratings_t.rating * ratings_t.similarity
    result = ratings_t.groupby(by = 'title').agg({'sim_rating' : 'sum', 'similarity' : 'sum'})
    result['mean_rating'] = result.sim_rating / result.similarity
    movie_ratings.loc[titles_na_critic, userID] = result.mean_rating

In [86]:
names = ratings.critic.tolist()

In [88]:
for userID in names :
    generateRecommendations(userID)

In [92]:
# 결측치를 모두 채운 형태
movie_ratings

,Claudia Puig,Gene Seymour,Jack Matthews,Lisa Rose,Mick LaSalle,Toby
title,,,,,,
Just My Luck,3.00000,1.5,2.530981,3.0,2.0,2.530981
Lady in the Water,2.83255,3.0,3.000000,2.5,3.0,2.832550
Snakes on a Plane,3.50000,3.5,4.000000,3.5,4.0,4.500000
Superman Returns,4.00000,5.0,5.000000,3.5,3.0,4.000000
The Night Listener,4.50000,3.0,3.000000,3.0,3.0,3.347790
You Me and Dupree,2.50000,3.5,3.500000,2.5,2.0,1.000000


In [91]:
# 소수점 둘째자리에서 반올림 해줌
np.round(movie_ratings, 1)

,Claudia Puig,Gene Seymour,Jack Matthews,Lisa Rose,Mick LaSalle,Toby
title,,,,,,
Just My Luck,3.0,1.5,2.5,3.0,2.0,2.5
Lady in the Water,2.8,3.0,3.0,2.5,3.0,2.8
Snakes on a Plane,3.5,3.5,4.0,3.5,4.0,4.5
Superman Returns,4.0,5.0,5.0,3.5,3.0,4.0
The Night Listener,4.5,3.0,3.0,3.0,3.0,3.3
You Me and Dupree,2.5,3.5,3.5,2.5,2.0,1.0
